## Why use SAP Generative Hub's Orchestration Service

Access to generative AI models often needs to be combined with other functionalities. These include:

- Prompting models using scenario-specific **templates** from a Prompt Repository.

- Ensuring compliance with AI Ethics and Responsible AI through **Content Filtering**.

- Maintaining data privacy by using **Data Masking** techniques.

- Enhancing models with business context through **Retrieval-Augmented Generation (RAG)** .

You need a service for coordinating and managing the deployment, integration, and interaction of various AI components.

This can be an erroneous and time-consuming process leading to complex and redundant code and workflows.

This is where orchestration services can be helpful.

> ![need for orchestration](./images/orchestration-1-need.png)

AI orchestration is the process of coordinating and managing how various AI components are deployed, integrated, and interact within a system or workflow.

Orchestration services and workflows in generative AI hub are useful in creating sophisticated workflows without complex code.

We can use orchestration services for different foundational models without changing the client code.

This approach **reduces maintenance, enhances control, and optimizes efficiency, helping teams focus on innovation rather than integration**. It helps you design powerful AI workflows visually and bring your AI vision to life faster with modular capabilities and intuitive interfaces.

In addition, it allows seamless integration and management of diverse components like data pipelines, AI models, and prebuilt modules (content filtering, data masking). It also ensures efficient execution of multiple AI models, optimizes computational resources, and automates the end-to-end AI lifecycle.

> ![need for orchestration](./images/orchestration-2-benefits.png)

## Use Case - Supplier Risk Analysis

At **BestRun SE**, the company recently kicked off a new project and tasked their supply chain planner with identifying the optimal supplier among several promising candidates. The planner consolidated supplier data from various sources into a single text file to facilitate a thorough risk analysis. Now, the planner must evaluate supplier risk by examining key performance indicators such as reliability, quality, and market trends to make an informed decision.

This exercise demonstrates how to leverage SAP Generative AI Hub's orchestration service to build an integrated solution for supplier risk analysis. 

The solution is designed to assess supplier performance by analyzing key metrics such as quality, pricing, and delivery times while also incorporating additional contextual information like industry trends and historical evaluations. 

Sensitive supplier information is securely protected through data masking, ensuring that personal details (e.g., contact name, email, and phone) are anonymized. 

The orchestration service streamlines the process by coordinating between multiple components—data ingestion from files, simulated grounding via embedding, and the execution of an advanced language model—enabling organizations to gain actionable insights into supplier risk and overall procurement strategy.

## Step 1. Setup Python Environment

The following installation only needs to be done once.

### Install required Python packages. 

**Note**: You only need to install the following packages once. It could take somewhere between 1 - 3 minutes to complete the installation. 

In [ ]:
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install ipywidgets --break-system-packages
%pip install python-dotenv --break-system-packages

### Now, before you continue, please restart the kernel by clicking on the **Restart** button on the top

### Import Environment and Orchestraion Packages

In [ ]:
# --- import json package to read input file ---
import json

# --- import packages to read environment variable
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

### Make sure to import the following SAP provided Orchestration Service packages 

In [2]:
# --- Set Up Orchestration Service packages ---
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.llm import LLM

from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, ProfileEntity

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService

### Connect with SAP AI Core Instance

In [ ]:
# Get the AI Core URL from environment variables
URL = os.getenv('AICORE_AUTH_URL')
# Get the AI Core client ID from environment variables
CLIENT_ID = os.getenv('AICORE_CLIENT_ID')
# Get the AI Core client secret from environment variables
CLIENT_SECRET = os.getenv('AICORE_CLIENT_SECRET')
# Get the AI Core client ID from environment variables
RESOURCE_GROUP = os.getenv('AICORE_RESOURCE_GROUP')
# Get the AI Core client secret from environment variables
API_URL = os.getenv('AICORE_BASE_URL')

# Determine the user's home directory
home_dir = os.path.expanduser('~')
aicore_dir = os.path.join(home_dir, '.aicore')

# Create the .aicore directory if it doesn't exist
os.makedirs(aicore_dir, exist_ok=True)

# Define the configuration data
config_data = {
    "AICORE_AUTH_URL": URL,
    "AICORE_CLIENT_ID": CLIENT_ID,
    "AICORE_CLIENT_SECRET": CLIENT_SECRET,
    "AICORE_RESOURCE_GROUP": RESOURCE_GROUP,
    "AICORE_BASE_URL": API_URL
}

# Path to the config.json file
config_path = os.path.join(aicore_dir, 'config.json')

# Write the config data to config.json
with open(config_path, 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print(f"Your resource group is {RESOURCE_GROUP}")

## Step 2: Configure data masking as per the use case's needs.  
**Note:**  This is an optional step. You need to decide if this step is needed per your own use case. 

In [5]:
# Create a proper data masking configuration
data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON
            ]
        )
    ]
)

## Step 3: Read and ground the data
**Note:** This step is simplified for the exercise. In the actual implementation, data preparation can be complex and time consuming.  Our focus of this exercise is the key steps of designing and running an orchestration flow.   

In [6]:
# --- Read Supplier Data from File ---
with open('data/supplier_data.json', 'r', encoding='utf-8') as f:
    sample_supplier_data = f.read()


In [7]:
# --- Read Grounding Data from File ---
with open('data/grounding_data.json', 'r', encoding='utf-8') as f:
    sample_grounding_data = f.read()


In [8]:
# Simulate Embedding for grounding data

# Note, we simplified text data embedding on purpose in module 1 of this exercise
# We will introduce how to properly setup and run embedding in modoule 2, using SAP HANA Cloud Vector Engine.

def embed_text(text):
    """
    Simulate text embedding.
    In a real system, this would output a vector representation.
    Here we simply return the text prefixed to indicate it's 'embedded'.
    """
    # The returned 'vector' is just a placeholder string.
    return f"[Embedded Vector Representation of]: {text}"

def retrieve_relevant_context(embedded_grounding, query, top_k=1):
    """
    Simulate a similarity search that retrieves the most relevant grounding context.
    In a real implementation, you would perform a vector similarity search.
    Here we simply return the original embedded grounding data.
    """
    # For simplicity, we assume the entire embedded grounding is relevant.
    return embedded_grounding

In [9]:
# Simulate embedding and retrieval of grounding context 
embedded_grounding = embed_text(sample_grounding_data)
# Simulate a retrieval step based on a query (here we use a dummy query for demonstration)
query = "What are the industry trends and internal evaluations affecting supplier performance?"
relevant_grounding_context = retrieve_relevant_context(embedded_grounding, query)

## Step 4: Setup orchestration flow
**Note:**   This includes  
4.1 Create the prompt template. By defining prompt template, you can easily change your message without changing the code structure.  
4.2 Configure the llm to be used  
4.3 Create the orchestration configuration

In [10]:
# Define a prompt template that now includes grounding context.
template = Template(
    messages=[
        SystemMessage("You are an AI procurement assistant specialized in supplier evaluation. Note: Sensitive supplier information has been masked."),
        UserMessage(
            "Given the following supplier data and additional grounding context, analyze and recommend the best supplier based on quality, price, and delivery performance. \n\nSupplier Data: {{?supplier_data}}\n\nGrounding Context: {{?grounding_context}}"
        )
    ]
)

In [11]:
# Configure the Language Model (LLM) for processing.
llm = LLM(
    name="gpt-4o",  # Replace with your model name if needed
    version="latest",
    parameters={"max_tokens": 350, "temperature": 0.3}
)

In [12]:
# Create the orchestration configuration with the data masking config.
config = OrchestrationConfig(
    template=template,
    llm=llm,
    data_masking=data_masking  # Use the configuration object instead of a boolean
)

## Step 5: Instantiate the orechestrateion service

In [13]:
# Get the Supplier Sourcing API URL from environment variables
YOUR_API_URL = os.getenv('SUPPLIER_SOURCING_ENDPOINT')
if not YOUR_API_URL:
    raise ValueError("SUPPLIER_SOURCING_ENDPOINT is not set in your .env file.")

# Now use YOUR_API_URL to instantiate the orchestration service
orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)

## Step 6: Run the orchestration workflow

In [14]:
# 6.1 Run the orchestration workflow by providing both the supplier data and the retrieved grounding context.
result = orchestration_service.run(template_values=[
    TemplateValue(name="supplier_data", value=sample_supplier_data),
    TemplateValue(name="grounding_context", value=relevant_grounding_context)
])

In [ ]:
# 6.2 Print the recommendation output.
print("Supplier Sourcing Recommendation with Data Masking and Grounding (Embedded):")
print(result.orchestration_result.choices[0].message.content)

## Key Learnings:  

The exercise highlights several crucial steps to create a robust orchestration solution using the SAP Generative AI Hub SDK.  

When you build your own AI workflow using the orchestration service, try to adopt the following key steps and apply them to your specific use case. 

1. **Initialize the Orchestration Service**: Before you use the generative-AI-hub-SDK, set up a virtual deployment of the Orchestration Service. After deployment, you'll receive a unique endpoint URL.

> ```python
>   YOUR_API_URL = "..."
> ```

<br>

2. **Define the Template and Default Input Values**: You can use the URL to access orchestration service.

For example: 

> ```python
>    from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
>    from gen_ai_hub.orchestration.models.template import Template, TemplateValue
>
>    template = Template(
>    messages=[
>       SystemMessage("You are a helpful translation assistant."),
>       UserMessage(
>            "Translate the following text to {{?to_lang}}: {{?text}}"
>       ),
>    ],
>    defaults=[
>        TemplateValue(name="to_lang", value="German"),
>    ],
>   )
> ```

<br>

3. **Define the LLM**

For example:

> ```python
>   from gen_ai_hub.orchestration.models.llm import LLM
>
>   llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})
> ```

<br>

4. **Create the orchestration configuration**

For example: 

> ```python
>   from gen_ai_hub.orchestration.models.config import OrchestrationConfig
>
>   config = OrchestrationConfig(
>    template=template,
>    llm=llm,
>   )
>
> ```

<br>

5. **Run the orchestration request**

For example:

> ```python
>   from gen_ai_hub.orchestration.service import OrchestrationService
>
>   orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)
>   result = orchestration_service.run(template_values=[
>   TemplateValue(name="text", value="The Orchestration Service is working!")
>   ])
>   print(result.orchestration_result.choices[0].message.content)
> ```

